In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.datasets as datasets
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import numpy as np

Load the IMDB dataset


In [2]:
(x_train, y_train), (x_test, y_test) = datasets.imdb.load_data(num_words=10000)

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do t

In [3]:
len(x_train)

25000

In [5]:
[len(x_train[i]) for i in range(10)]

[218, 189, 141, 550, 147, 43, 123, 562, 233, 130]

# Preprocessing API

In [6]:
import tensorflow.keras.preprocessing.sequence as sequence

In [7]:
x_padded = sequence.pad_sequences(x_train, maxlen=200)

In [8]:
[len(x_padded[i]) for i in range(10)]

[200, 200, 200, 200, 200, 200, 200, 200, 200, 200]

The padded sequences are stored in a tensor.

In [10]:
x_padded[:10]

array([[   5,   25,  100, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [  42, 1134,    6, ...,   72,   33,   32],
       [5533,   15,    4, ...,   28,  126,  110],
       [   0,    0,    0, ...,    7,   43,   50]], dtype=int32)

# The Embedding Layer

Construct an embedding layer

In [12]:
V = 10000
emb_dim = 20

embedding = layers.Embedding(V, emb_dim)

We can use the embedding layer to map ordinals to vectors.

In [16]:
# This is a batch of size 1,
# the sequence is three ordinals: 1, 2, 2
input_seq = np.array([[1,2,2]])

embedding(input_seq)

<tf.Tensor: shape=(1, 3, 20), dtype=float32, numpy=
array([[[ 0.01752738, -0.03314637,  0.01473473,  0.0061217 ,
          0.03070916,  0.01097447, -0.02551485,  0.04575075,
         -0.04992241, -0.03964483, -0.01084699,  0.02344653,
          0.04034713, -0.04007636,  0.00895436,  0.02949199,
          0.04813573,  0.0168754 ,  0.01897048, -0.02920285],
        [ 0.04358936,  0.02829284,  0.04363424,  0.0353098 ,
         -0.04069038,  0.04126066,  0.0076711 , -0.03629258,
         -0.00337466,  0.04948654, -0.03392481,  0.03110505,
          0.04464323,  0.01402121,  0.031125  ,  0.01389359,
         -0.03116088,  0.03402023,  0.03258195, -0.01341359],
        [ 0.04358936,  0.02829284,  0.04363424,  0.0353098 ,
         -0.04069038,  0.04126066,  0.0076711 , -0.03629258,
         -0.00337466,  0.04948654, -0.03392481,  0.03110505,
          0.04464323,  0.01402121,  0.031125  ,  0.01389359,
         -0.03116088,  0.03402023,  0.03258195, -0.01341359]]],
      dtype=float32)>

In [20]:
output_vecs = embedding(input_seq)

print(input_seq.shape, "=>", output_vecs.shape)

(1, 3) => (1, 3, 20)


The embedding layer has a model parameter which is the embedding matrix.

In [19]:
embedding.get_weights()[0].shape

(10000, 20)

# Recurrent Neural Network Layer: SimpleRNN

Construct a simple RNN.

In [21]:
state_dim = 7
rnn = layers.SimpleRNN(state_dim)

In [22]:
# start with a sequence of ordinals:
input_seq = np.array([[1,2,2], [1,12,0]])
print("input_seq:", input_seq.shape)

# embed them into our vector space of dim 20.
embedded_vectors = embedding(input_seq)
print("embdded_vectors:", embedded_vectors.shape)

# map the embedded vector sequence into a single state vector
state_vector = rnn(embedded_vectors)
print("state_vector:", state_vector.shape)

input_seq: (2, 3)
embdded_vectors: (2, 3, 20)
state_vector: (2, 7)


# Finally, we can map the state to category

Construct a dense layer with sigmoid activation.

In [25]:
dense = layers.Dense(1, activation='sigmoid')

In [26]:
# start with a sequence of ordinals:
input_seq = np.array([[1,2,2], [1,12,0]])
print("input_seq:", input_seq.shape)

# embed them into our vector space of dim 20.
embedded_vectors = embedding(input_seq)
print("embdded_vectors:", embedded_vectors.shape)

# map the embedded vector sequence into a single state vector
state_vector = rnn(embedded_vectors)
print("state_vector:", state_vector.shape)

# map the state vector to a probability of 0, 1.
category = dense(state_vector)
print(category)

input_seq: (2, 3)
embdded_vectors: (2, 3, 20)
state_vector: (2, 7)
tf.Tensor(
[[0.4951419 ]
 [0.49233618]], shape=(2, 1), dtype=float32)


# Building an end-to-end network

We will use the functional API to build the network.

In [28]:
maxlen = 200
inputs = layers.Input(shape=(maxlen))
x = embedding(inputs)
x = rnn(x)
output = dense(x)

In [30]:
model = models.Model(inputs=inputs, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 20)           200000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 7)                 196       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 8         
Total params: 200,204
Trainable params: 200,204
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [33]:
# Train the model

model.fit(x_padded, y_train, epochs=5, batch_size=128, validation_split=0.2)

Epoch 1/5
157/157 [==============================] - 11s 62ms/step - loss: 0.6247 - acc: 0.6744 - val_loss: 0.4891 - val_acc: 0.8186
Epoch 2/5
157/157 [==============================] - 10s 62ms/step - loss: 0.4365 - acc: 0.8461 - val_loss: 0.4303 - val_acc: 0.8230
Epoch 3/5
157/157 [==============================] - 9s 60ms/step - loss: 0.3517 - acc: 0.8771 - val_loss: 0.4650 - val_acc: 0.7932
Epoch 4/5
157/157 [==============================] - 10s 62ms/step - loss: 0.3289 - acc: 0.8783 - val_loss: 0.3875 - val_acc: 0.8374
Epoch 5/5
157/157 [==============================] - 10s 63ms/step - loss: 0.2664 - acc: 0.9078 - val_loss: 0.3883 - val_acc: 0.8452
